# Test Cache in Jupyter Notebook

In [ ]:
import joblib

print(joblib)

import helpers.cache as hcac

In [ ]:
import helpers.dbg as dbg

# Define computation function

This function will be subjected to cache.

In [ ]:
def computation_function(a, b):
    out = a + b
    print("Sum: %s + %s = %s" % (a, b, out))
    return out


inputs = (1, 2)
exp_output = 3

computation_function(*inputs)

In [ ]:
def computation_function(a, b):
    # hello
    # assert 0
    out = a * b
    print("Multiplication: %s * %s = %s" % (a, b, out))
    return out


inputs = (1, 2)
exp_output = 2

computation_function(*inputs)

## LRU Cache

In [ ]:
def hello():
    print("hello")


print(id(hello))

In [ ]:
import functools

mem_func = functools.lru_cache(maxsize=None)(hello)

mem_func()

In [ ]:
mem_func = functools.lru_cache(maxsize=None)(hello)

mem_func()

In [ ]:
def hello():
    print("hello")


print(id(hello))

In [ ]:
def hello():
    assert 0
    print("good bye")


print(id(hello))

In [ ]:
# hello()
print(id(hello))
mem_func()

In [ ]:
mem_func = functools.lru_cache(maxsize=None)(hello)

mem_func()

## Joblib cache

In [ ]:
import joblib

disk_cache = joblib.Memory("./joblib.cache", verbose=0, compress=1)
disk_cache.clear()

In [ ]:
def hello():
    print("hello")


print(id(hello))

cached_hello = disk_cache.cache(hello)

print(id(cached_hello))

In [ ]:
cached_hello()

In [ ]:
cached_hello()

In [ ]:
def hello():
    print("hello")

In [ ]:
cached_hello()

In [ ]:
def hello():
    print("good bye")


cached_hello = disk_cache.cache(hello)

In [ ]:
cached_hello()

In [ ]:
def hello():
    print("good bye")


cached_hello = disk_cache.cache(hello)

cached_hello()

## Memory cache

In [ ]:
def computation_function(a, b):
    out = a + b
    print("Sum: %s + %s = %s" % (a, b, out))
    return out


inputs = (1, 2)
exp_output = 3

In [ ]:
memory_cached_computation = hcac.Cached(
    computation_function, use_mem_cache=True, use_disk_cache=False
)

dbg.dassert_eq(memory_cached_computation(*inputs), exp_output)
dbg.dassert_eq(memory_cached_computation.get_last_cache_accessed(), "no_cache")

dbg.dassert_eq(memory_cached_computation(*inputs), exp_output)
dbg.dassert_eq(memory_cached_computation.get_last_cache_accessed(), "mem")

print("memory caching checks passed")

In [ ]:
def computation_function(a, b):
    # hello
    # assert 0
    out = a * b
    print("Multiplication: %s * %s = %s" % (a, b, out))
    return out


inputs = (1, 2)
exp_output = 2

dbg.dassert_eq(memory_cached_computation(*inputs), exp_output)
print(memory_cached_computation.get_last_cache_accessed())
dbg.dassert_eq(memory_cached_computation.get_last_cache_accessed(), "mem")

In [ ]:
memory_cached_computation._func.__code__
memory_cached_computation._func.co_filename

In [ ]:
computation_function.__code__

## Disk cache

In [ ]:
disk_cached_computation = hcac.Cached(
    computation_function, use_mem_cache=False, use_disk_cache=True
)

disk_cached_computation.clear_cache("disk")

dbg.dassert_eq(disk_cached_computation(*inputs), exp_output)
dbg.dassert_eq(disk_cached_computation.get_last_cache_accessed(), "no_cache")

dbg.dassert_eq(disk_cached_computation(*inputs), exp_output)
dbg.dassert_eq(disk_cached_computation.get_last_cache_accessed(), "disk")

print("disk caching checks passed")

In [ ]:
print(disk_cached_computation._execute_func_from_disk_cache.func)

from joblib.func_inspect import get_func_code

print(
    get_func_code(disk_cached_computation._execute_func_from_disk_cache.func)[0]
)
dbg.dassert_eq(disk_cached_computation(*inputs), exp_output)
dbg.dassert_eq(disk_cached_computation.get_last_cache_accessed(), "disk")

## Full cache

In [ ]:
fully_cached_computation = hcac.Cached(
    computation_function, use_mem_cache=True, use_disk_cache=True
)

fully_cached_computation.clear_cache("disk")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "no_cache")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

print("Clear mem cache")
fully_cached_computation.clear_cache("mem")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "disk")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

print("full caching checks passed")

In [ ]:
dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

In [ ]:
# This should fail all the times, because we clear the memory cache.
fully_cached_computation.clear_cache("mem")
dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")